Data Wrangle OpenStreetMaps Data
=================

Step One - Finish Lession 6
-------------------------

Done.

Step Two - Review the Rubric and Sample Project
--------------------------------------------

Done.

Step Three - Choose Your Map Area
-------------------------------

We use [Map Zen](https://mapzen.com/data/metro-extracts) to download the preselected metro area of San Francisco, California. After uncompressing, the dataset size is 648.9MB.

Step Four - Process your Dataset
------------------------------

### 4.1 Audit the dataset

#### Sanity check on element types

As a sanity check, we go through the dataset to see list all element types, which should contain the following:
  * `osm`: top-level root node
  * `node`, `way` and `relation`: instances of data primitives
  * `tag`: a general purpose node for key/value pair.
  * `nd`: used inside `way`s to reference a `node` element.
  * `member`: used inside a `relation`.

In [1]:
import xml.etree.cElementTree as ET

xml_file = "data/san-francisco_california.osm"
elem_types = set()
for _, elem in ET.iterparse(xml_file):
    # If the element type is not seen before, add it to the set.
    if elem.tag not in elem_types:
        elem_types.add(elem.tag)

print elem_types

set(['node', 'nd', 'bounds', 'member', 'tag', 'relation', 'way', 'osm'])


The only "unexpected" element type is `bounds`, which occurs only once in the dataset to indicate the bounding box of this dataset.

#### Street address types

Next we try to see different address types of all elements, and how many times each address type appears in the dataset. The address type is found from the `<tag k="addr:XXX" v="xxxxxx">` tag.

In [2]:
import operator

address_types = {}
for _, tag in ET.iterparse(xml_file):
    if tag.tag == "tag" and tag.attrib['k'].startswith("addr:"):
        k = tag.attrib['k'][5:]
        address_types.setdefault(k, 0)
        address_types[k] += 1

for k,v in sorted(address_types.items(), key=operator.itemgetter(1), reverse=True):
    print k, ':', v

housenumber : 47790
city : 44022
street : 40586
postcode : 9503
state : 3224
country : 1262
county : 374
interpolation : 317
housename : 293
unit : 56
full : 43
paloaltoca_id : 43
housenumber:source : 3
street:source : 2
floor : 2
suite : 2
pier : 1
province : 1
1:housenumber : 1
place : 1
door : 1


We can see the majority of address types are `housenumber`(47790), `city`(44022), `street`(40586) and `postcode`(9503), and the relatively rare address types are `housenumber:source`, `street:source`, `floor`, `suite`, `pier`, `province`, `1:housenumber`, `place`, `door`, which happens only once or twice in the dataset.

#### Street names

After having the street address types, we try to audit all the street name in the dataset. Street name is extracted as the last word of `<tag k="addr:street" v="xxxxxx">` tag. We also keep a small number of examples for each street name type during the auditing.

In [3]:
from collections import defaultdict
import re

street_name_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

street_names = defaultdict(int)
street_name_examples = defaultdict(list)

for _, tag in ET.iterparse(xml_file):
    if tag.tag == "tag" and tag.attrib['k'] == "addr:street":
        street = tag.attrib['v']
        match = street_name_re.search(street)
        if match:
            name = match.group().lower()
            street_names[name] += 1
            if street_names[name] <= 3:
                street_name_examples[name].append(street)

for k,v in sorted(street_names.items(), key=operator.itemgetter(1), reverse=True):
    print k, ':', v, '[', ', '.join(street_name_examples[k]), (v>=3 and ",...]" or ']')

street : 15350 [ Haight Street, Haight Street, Haight Street ,...]
avenue : 13322 [ College Avenue, College Avenue, Keeler Avenue ,...]
road : 3067 [ Gouldin Road, Primrose Road, Shasta Road ,...]
drive : 2070 [ Laird Drive, Middlefield Drive, Skyline Drive ,...]
way : 1649 [ Bancroft Way, Martin Luther King Jr Way, Orinda Way ,...]
court : 824 [ Prescott Court, Prescott Court, Prescott Court ,...]
boulevard : 823 [ West Hillsdale Boulevard, Fremont Boulevard, Treat Boulevard ,...]
lane : 774 [ Laurel Lane, Laurel Lane, Arcade Lane ,...]
place : 431 [ William Saroyan Place, Romolo Place, Bayview Place ,...]
real : 421 [ South El Camino Real, El Camino Real, El Camino Real ,...]
broadway : 397 [ Broadway, Broadway, Broadway ,...]
alameda : 376 [ The Alameda, The Alameda, The Alameda ,...]
circle : 167 [ Holly Park Circle, Columbia Circle, Columbia Circle ,...]
ave : 87 [ Floribunda Ave, Paloma Ave, Grant Ave ,...]
st : 86 [ Park St, S Delaware St, 16th St ,...]
terrace : 85 [ Greenwood 

From the audit above, we see the common street names (ones that I expected to see) are:
  * street
  * avenue
  * road
  * way
  * drive
  * boulevard
  * broadway
  * lane
  * plaza
  * square
  * parkway
  * highway
  * walk
    
Some common abbreviations are:

  * st => street
  * ave => avenue
  * plz => plaza
  * blvd => boulevard
  * st => street
  * dr => drive
  * rd => road
  * ave. => avenue
  * dr. => drive
  * hwy => highway
  * blvd. => boulevard

There are also some valid but uncommon street names (ones that I didn't expect to see):

  * real : [ South El Camino Real, El Camino Real ,...]
  * court : [ Prescott Court, Prescott Court ,...]
  * place : [ William Saroyan Place, Romolo Place ,...]
  * alameda : [ The Alameda, The Alameda ,...]
  * circle : [ Holly Park Circle, Columbia Circle ,...]
  * center : [ Fort Mason Center, Seramonte Center ,...]
  * path : [ Parnassus Path, Parnassus Path ,...]
  * terrace : [ Greenwood Terrace, Hawthorne Terrace ,...]
  * embarcadero : [ The Embarcadero, The Embarcadero ,...]
  
The rest street names are either very uncommon erroneously formatted (e.g. `leimert : [ Leimert, Leimert ]`, `rock : [ Mission Rock ]`, `410 : [ 18th Street Ste 410 ]`, `h : [ Avenue H ]` etc.).

### 4.2 Clean the dataset and convert to JSON

#### Clean street names

Based on audit above, we clean the street name with the following rules:
  * keep the common street names, both common ones (e.g. `street`, `road`) and the uncommone ones (e.g. `real`, `embarcadero`);
  * convert the abbreviation to full name, e.g. `st => street`, `blvd => boulevard`;
  * drop the very uncommon or erroneously formatted names, e.g. `410 [ 18th Street Ste 410 ]`, `rock : [ Mission Rock ]`.

In [4]:
common_types = ["street", "avenue", "road", "way", "drive",
                "boulevard", "broadway", "lane", "plaza", "square",
                "parkway", "highway", "walk"]
uncommon_types = ["real", "court", "place", "alameda", "circle",
                  "center", "path", "terrace", "embarcadero"]
abbreviations = {"st": "street",
                 "ave": "avenue",
                 "plz": "plaza",
                 "blvd": "boulevard",
                 "st": "street",
                 "dr": "drive",
                 "rd": "road",
                 "ave.": "avenue",
                 "dr.": "drive",
                 "hwy": "highway",
                 "blvd.": "boulevard"}

def clean_street_name(street_name):
    """Clean the street name:
    * for valid street types (common or nuncommon), name is unchanged;
    * abbreviated street type will be expanded;
    * for all other cases (invalid or erroneous), return `None`."""
    m = street_name_re.search(street_name)
    if m:
        street_type = m.group().lower()
        if street_type in common_types or street_type in uncommon_types:
            # Valid street type, no change need to be made.
            return street_name
        if street_type in abbreviations:
            # Abbreviated street type need to be expanded.
            return street_name_re.sub(abbreviations[street_type], street_name)
    # In all other cases (invalid or erroneous format), return None.
    return None

#### Address dictionary

We create an address dictionary from the tags of an xml element as follows

In [5]:
def get_element_address(element):
    """For a given element in the xml, create an `address` dictionary
    with the following fields: `housenumber`, `street`, `city` and `postcode`,
    and the `street` field will be cleaned by `clean_street_name`."""
    address = {}
    for tag in element:
        for k in ["housenumber", "city", "postcode"]:
            if tag.attrib.get('k') == "addr:" + k:
                address[k] = tag.attrib['v']
        if tag.attrib.get('k') == "addr:street":
            street_name = clean_street_name(tag.attrib['v'])
            if street_name:
                address["street"] = street_name
    return address

#### Created dictionary

Similary, we create a `created` dictionary from the attribute of each element (`node`, `way` or `relation`).

In [6]:
def get_element_created(elem):
    """Get a 'created' dictionary from an element, that has the following keys:
    'version', 'changeset', 'timestamp', 'uid', 'user'."""
    created = {}
    for k in ["version", "changeset", "timestamp", "uid", "user"]:
        if k in elem.attrib:
            created[k] = elem.attrib[k]
    return created

#### Convert the xml element to dictionary

Now we are ready to convert each xml element (`node`, `way` or `relation`) into a "json-ready" dictionary.

In [7]:
def xml_to_json_helper(elem, dst):
    """A helper function for converting element to json-ready dictionary.
    This will put the common attribute, e.g. `created`, `address`, and
    tags into the `dst` dictionary."""
    dst["created"] = get_element_created(elem)
    dst["address"] = get_element_address(elem)
    # Append non-address tags.
    for tag in elem:
        if 'k' in tag.attrib and not tag.attrib['k'].startswith('addr:'):
            dst[tag.attrib['k']] = tag.attrib['v']
    return dst


def shape_node(elem):
    """Convert a `node` element to a json-ready dictionary."""
    node = {"element_type": "node"}
    xml_to_json_helper(elem, node)
    # `id` and `coord` attributes.
    node["id"] = elem.attrib["id"]
    node["coord"] = [float(elem.attrib["lat"]), 
                     float(elem.attrib["lon"])]
    return node


def shape_way(elem):
    """Convert a `way` element to a json-ready dictionary."""
    way = {"element_type": "way"}
    xml_to_json_helper(elem, way)
    # node references.
    way["node_refs"] = []
    for nd in elem:
        if nd.tag == "nd":
            way["node_refs"].append(nd.attrib["ref"])
    return way


def shape_relation(elem):
    """Convert a `relation` element to a json-ready dictionary."""
    relation = {"element_type": "relation"}
    xml_to_json_helper(elem, relation)
    # members.
    relation["members"] = []
    for member in elem:
        if member.tag == "member":
            md = {}
            for k in ["type", "ref", "role"]:
                if k in member.attrib:
                    md[k] = member.attrib[k]
            relation["members"].append(md)
    return relation

Now we can iterate through the xml file, convert each top-level element (`node`, `way` or `relation`) to a json-ready dictionary and dump into a json file.

In [8]:
import json
import xml.etree.cElementTree as ET

xml_file = "data/san-francisco_california.osm"
json_file = xml_file + ".json"

with open(json_file, 'w') as f:
    for _, elem in ET.iterparse(xml_file):
        # Convert the element to json dictionary according to type.
        if elem.tag == "node":
            j = shape_node(elem)
        elif elem.tag == "way":
            j = shape_way(elem)
        elif elem.tag == "relation":
            j = shape_relation(elem)
        else:
            j = None
        # Dump the json dictionary to file.
        if j:
            f.write(json.dumps(j) + "\n")

## 4.3 Import into a MangoDB database and run queries

#### Import data and setup client

First we instantiate a mongodb daemon process, using a local `db/` directory as database path.

    mkdir -p `pwd`/db
    mongod -dbpath `pwd`/db

Then we can import the converted json file into the database using `mongoimport`.

    mongoimport --db openstreetmap --collection sanfrancisco --drop \
                 --file data/san-francisco_california.osm.json

We use `pymongo` to interact with the database. A client can be setup as follows:

In [10]:
from pymongo import MongoClient

client = MongoClient()
db = client.openstreetmap
collection = db.sanfrancisco

#### Running queries

Count number of `node`s, `way`s and `relation`s.

In [11]:
print "Number of <node>s:", collection.find({"element_type":"node"}).count()
print "Number of <way>s:", collection.find({"element_type":"way"}).count()
print "Number of <relation>s:", collection.find({"element_type":"relation"}).count()

Number of <node>s: 3018836
Number of <way>s: 324910
Number of <relation>s: 3322


Count number of distinct users, and show the top 10 users in  terms of number of contributed entries. From the results we can see the top 6 users contribute more than 100,0000 entries each, and the top contributor has 10 times more contributions than the number 7 contributor.

In [13]:
print "Number of distinct users:", len(collection.distinct("created.user"))

print "Top contributors:"
cursor = collection.aggregate([
        {"$group": {"_id": "$created.user", "count": {"$sum":1}}},
        {"$sort": {"count": -1}},
        {"$limit": 10}])
for c in cursor:
    print c

Number of distinct users: 1970
Top contributors:
{u'count': 730014, u'_id': u'ediyes'}
{u'count': 561447, u'_id': u'Luis36995'}
{u'count': 421017, u'_id': u'Rub21'}
{u'count': 337422, u'_id': u'oldtopos'}
{u'count': 137380, u'_id': u'KindredCoda'}
{u'count': 117709, u'_id': u'DanHomerick'}
{u'count': 72845, u'_id': u'nmixter'}
{u'count': 57892, u'_id': u'dchiles'}
{u'count': 53922, u'_id': u'oba510'}
{u'count': 52657, u'_id': u'brentengust'}


Check the most common amenties. We see that the number 1 is "parking", followed by "restaurant", which is quite respected in our San Francisco dataset.

In [14]:
cursor = collection.aggregate([
        {"$match": {"amenity": {"$exists":1}}},
        {"$group": {"_id":"$amenity", "count":{"$sum":1}}},
        {"$sort": {"count":-1}},
        {"$limit": 10}])
for c in cursor:
    print c

{u'count': 3641, u'_id': u'parking'}
{u'count': 2320, u'_id': u'restaurant'}
{u'count': 1316, u'_id': u'school'}
{u'count': 1086, u'_id': u'place_of_worship'}
{u'count': 836, u'_id': u'fire_hydrant'}
{u'count': 716, u'_id': u'cafe'}
{u'count': 640, u'_id': u'post_box'}
{u'count': 628, u'_id': u'bench'}
{u'count': 512, u'_id': u'fast_food'}
{u'count': 443, u'_id': u'bicycle_parking'}


As an additional exploration, we check the most common street names in the city. Note that the street names has been cleaned in our previous step. The results below show the number 1 common street name is "El Camino Real", which is a quite unique street name, followed by "Broadway", which is a more common name across United States.

In [15]:
cursor = collection.aggregate([
        {"$match": {"address.street": {"$exists":1}}},
        {"$group": {"_id": "$address.street", "count": {"$sum":1}}},
        {"$sort": {"count": -1}},
        {"$limit":10}])
for c in cursor:
    print c

{u'count': 408, u'_id': u'El Camino Real'}
{u'count': 392, u'_id': u'Broadway'}
{u'count': 337, u'_id': u'Alameda'}
{u'count': 334, u'_id': u'Church Street'}
{u'count': 322, u'_id': u'Jefferson Avenue'}
{u'count': 309, u'_id': u'Sanchez Street'}
{u'count': 296, u'_id': u'24th Street'}
{u'count': 296, u'_id': u'San Pablo Avenue'}
{u'count': 283, u'_id': u'Roosevelt Avenue'}
{u'count': 240, u'_id': u'Hudson Street'}


Step Five - Document your Work
----------------------------

Majority of documentation can be found in the previous section. Below are a few general thoughts about the dataset and the data wrangling process.

* First, I think openstreetmap is a great dataset with large amount of information, and I really appreciate the creators making it publicly available. However, during the analyzing, I also notice that the dataset is quite noisy, with quite a few entries that are invalid or erroneously formatted (e.g. the large number of street names that occur only once or twice are mostly erroneous). This is understandable because most of the data come from user input (which leads to the success of the dataset), but I believe a post scanning and trimming by professionals will make the dataset even stronger.
* One lesson I learned from this project is that auditing is a very important step for data wrangling. During the auditing, I can get a sense of data distributions, validity of assumptions I made, and possible errors from the dataset itself. It avoids a lot of errors and debugging in the cleaning and converting step.
* Lastly, I see the power of a database management system (mongodb in this case). After importing the data into mongodb, querying and analyzing is orders of mangnitude faster than similar processing the raw xml file. The time spent cleaning and importing the data into a database management system is ver